## Data Import

In [ ]:
import toolz as fp
import plotly.express as px
import pandas as pd
import numpy as np
from trello_music_board_processor import general_trello_music_pipeline, cards_to_dataframe, build_creation_at_data

json_file = "trello_music_board_data_backup_2023_11_20.json"
list_mapper, cards_by_list, cleaned_data = general_trello_music_pipeline(json_file)
cards_df = fp.pipe(
    cards_by_list,
    cards_to_dataframe,
    build_creation_at_data(board_id=cleaned_data["id"])
)
cards_df.shape

## Functions

In [ ]:
# Function to calculate RMSD
def calculate_rmsd(observed_counts, expected_frequencies):
    observed_probabilities = observed_counts / observed_counts.sum()
    rmsd = np.sqrt(((observed_probabilities - expected_frequencies) ** 2).mean())
    return rmsd

In [ ]:
def calculate_normalized_rmsd_list(cards_df):
    cards_df["stars"] = pd.to_numeric(cards_df.query("list_name != 'Want to'")["stars"], errors="coerce")
    cards_df.dropna(subset=["stars"], inplace=True)
    mean_rating = cards_df["stars"].mean()
    std_rating = cards_df["stars"].std()
    ratings = [1, 2, 3, 4, 5]
    expected_frequencies = [
        (1 / (std_rating * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((rating - mean_rating) / std_rating) ** 2)
        for rating in ratings
    ]
    # Normalize the expected frequencies so that they sum up to 1 (like a probability distribution)
    total_expected = sum(expected_frequencies)
    normalized_expected_frequencies = [freq / total_expected for freq in expected_frequencies]

    # Calculate the RMSD for each list and store it
    list_rmsds = {}
    for list_name in cards_df["list_name"].unique():
        list_cards = cards_df[cards_df["list_name"] == list_name]
        observed_counts = list_cards["stars"].value_counts().reindex(ratings, fill_value=0)
        rmsd = calculate_rmsd(observed_counts, normalized_expected_frequencies)
        list_rmsds[list_name] = rmsd
    # Normalize the RMSD values by dividing by the max RMSD
    max_rmsd = max(list_rmsds.values())
    normalized_rmsd_list = {list_name: rmsd / max_rmsd for list_name, rmsd in list_rmsds.items()}
    return normalized_rmsd_list

rmds_list = calculate_normalized_rmsd_list(cards_df)
# sort the list by rmsd (ascending)
rmds_list = {k: v for k, v in sorted(rmds_list.items(), key=lambda item: item[1], reverse=False)}

### General score distribution per list

In [ ]:
# Calculate the count and percentage of each star rating for each list
star_counts = cards_df.groupby(['list_name', 'stars']).size().reset_index(name='count').query("list_name != 'Want to'")
star_counts['percentage'] = star_counts.groupby('list_name')['count'].transform(lambda x: 100 * x / x.sum())
# Add RMSD as a column in the star_counts DataFrame
star_counts['rmsd'] = star_counts['list_name'].map(rmds_list)
# Convert RMSD to a string and prepare it as a label
star_counts['rmsd_label'] = 'RMSD: ' + star_counts['rmsd'].round(2).astype(str)
star_counts['percentage_text'] = star_counts['percentage'].round(1).astype(str) + '%'
# Now create the bar plot with RMSD labels
fig = px.bar(
    star_counts,
    x='stars',
    y='percentage',
    color='list_name',
    facet_col='list_name',
    category_orders={"stars": [1, 2, 3, 4, 5, 'No Rating']}, 
    title='Percentage of Cards per star rating for each list',
    labels={'percentage': 'Percentage of Cards', 'list_name': 'List Name', 'stars': 'Star Rating'},
    text='percentage_text'
)
# Update layout to adjust the text on the bars
fig.update_traces(texttemplate='%{text}', textposition='outside')


# Show the figure
fig.show()

In [ ]:
# create the year and month columns  - createdAt column is a string column
cards_df["created_at_aux"] = pd.to_datetime(cards_df["createdAt"])
cards_df["year"] = cards_df["created_at_aux"].dt.year.astype(str)
cards_df["month"] = cards_df["created_at_aux"].dt.month.astype(str)
cards_df["year_month"] = cards_df["created_at_aux"].dt.strftime("%Y-%m")

In [ ]:
def rel_pos_by(df, by="list_name"):
    df['rn'] = df.sort_values(by=[by, "pos"], ascending=[True, False]).groupby(by).cumcount()
    df_max_rn = df.groupby(by)['rn'].transform('max')
    df['max_rn'] = df_max_rn
    df['relative_rank'] = df['rn'] / df['max_rn']
    return df

In [ ]:
cards_df = cards_df.pipe(rel_pos_by)

In [ ]:
to_plot_df = cards_df.groupby(["year_month"]).size().reset_index().rename(columns={0: "count"}).sort_values(by="year_month", ascending=True)
fig = px.line(
    to_plot_df,
    x="year_month",
    y="count",
    title="Number of cards created per month"
)
fig.update_layout(width=800)

In [ ]:
import math
def order_card_by_and_cut(cards_df, cut=10, mode="best"):
    if mode == "best":
        ascending_order = [False, False]
    elif mode == "worst":
        ascending_order = [True, True]
    year_months = sorted(cards_df["year_month"].unique())
    outer = []
    for ym in year_months:
        curr_df = cards_df.query(f"year_month == '{ym}'").sort_values(by=["stars", "relative_rank"], ascending=ascending_order).head(cut)
        # print(f"Total cards in {ym}: {curr_df.shape[0]}")
        outer.append(curr_df[["year_month", "year", "month", "name", "stars", "relative_rank"]])
    outer_df = pd.concat(outer)
    # let's trim the name to be at most 30 characters
    outer_df["name_abr"] = outer_df["name"].apply(lambda x: x[:30] + "..." if len(x) > 30 else x)
    return outer_df

### Top 10 charts by Year and Month

In [ ]:
outer_df = order_card_by_and_cut(cards_df, cut=10, mode="best")
for year in sorted(cards_df["year"].unique()):
    curr_year_df = outer_df.query(f"year == '{year}'")
    months = curr_year_df.sort_values(by=["year_month"], ascending=True)["year_month"].unique()
    print(f"[TOP 10 CHARTS] - YEAR {year}\n\n")
    for ym in months:
        print(f"-------------------- [{ym}] --------------------")
        curr_year_month_df = curr_year_df.query(f"year_month == '{ym}'").sort_values(by=["stars", "relative_rank"], ascending=[False, False])
        (
            curr_year_month_df[['name', 'stars']]
                .head(10)
                .reset_index(drop=True)
                .reset_index()
                .rename(columns={"index": "RANK"})
                .assign(RANK=lambda x: x.RANK + 1)
                .apply(lambda x: print(f"\t {int(x['RANK'])}. {x['name']} ({(int(x['stars']))} stars)"), axis=1)
        )


### Top 10 worst charts by Year and Month

In [ ]:
outer_df = order_card_by_and_cut(cards_df, cut=10, mode="worst")
for year in sorted(cards_df["year"].unique()):
    curr_year_df = outer_df.query(f"year == '{year}'")
    months = curr_year_df.sort_values(by=["year_month"], ascending=True)["year_month"].unique()
    print(f"[TOP 10 WORST CHARTS] - YEAR {year}\n\n")
    for ym in months:
        print(f"-------------------- [{ym}] --------------------")
        curr_year_month_df = curr_year_df.query(f"year_month == '{ym}'").sort_values(by=["stars", "relative_rank"], ascending=[True, True])
        (
            curr_year_month_df[['name', 'stars']]
                .head(10)
                .reset_index(drop=True)
                .reset_index()
                .rename(columns={"index": "RANK"})
                .assign(RANK=lambda x: x.RANK + 1)
                .apply(lambda x: print(f"\t {int(x['RANK'])}. {x['name']} ({(int(x['stars']))} stars)"), axis=1)
        )

### Top 10 charts by Year

In [ ]:
mode = "best"
cut = 10
if mode == "best":
    ascending_order = [False, False]
elif mode == "worst":
    ascending_order = [True, True]
def grouped_ordering(g):
    return g.sort_values(by=["stars", "relative_rank"], ascending=ascending_order)[["year_month", "year", "month", "name", "stars", "relative_rank"]].head(cut)
outer_df = cards_df.groupby("year", group_keys=False).apply(grouped_ordering).reset_index(drop=True)
for year in sorted(outer_df["year"].unique()):
    curr_year_df = outer_df.query(f"year == '{year}'")
    print(f"[TOP 10 BEST CHARTS] - YEAR {year}\n")
    (
        curr_year_df[['name', 'stars']]
            .reset_index(drop=True)
            .reset_index()
            .rename(columns={"index": "RANK"})
            .assign(RANK=lambda x: x.RANK + 1)
            .apply(lambda x: print(f"\t {int(x['RANK'])}. {x['name']} ({(int(x['stars']))} stars)"), axis=1)
    )
    print("")

### Top 10 worst charts by Year

In [ ]:
mode = "worst"
cut = 10
if mode == "best":
    ascending_order = [False, False]
elif mode == "worst":
    ascending_order = [True, True]
def grouped_ordering(g):
    return g.sort_values(by=["stars", "relative_rank"], ascending=ascending_order)[["year_month", "year", "month", "name", "stars", "relative_rank"]].head(cut)
outer_df = cards_df.groupby("year", group_keys=False).apply(grouped_ordering).reset_index(drop=True)
for year in sorted(outer_df["year"].unique()):
    curr_year_df = outer_df.query(f"year == '{year}'")
    print(f"[TOP 10 WORST CHARTS] - YEAR {year}\n")
    (
        curr_year_df[['name', 'stars']]
            .reset_index(drop=True)
            .reset_index()
            .rename(columns={"index": "RANK"})
            .assign(RANK=lambda x: x.RANK + 1)
            .apply(lambda x: print(f"\t {int(x['RANK'])}. {x['name']} ({(int(x['stars']))} stars)"), axis=1)
    )
    print("")

### Top 20 by genre

In [ ]:
mode = "best"
cut = 20

if mode == "best":
    ascending_order = [False, False]
elif mode == "worst":
    ascending_order = [True, True]
def grouped_ordering(g):
    return g.sort_values(by=["stars", "relative_rank"], ascending=ascending_order)[["year_month", "year", "month", "name", "stars", "relative_rank", "list_name"]].head(cut)
outer_df = cards_df.groupby("list_name", group_keys=False).apply(grouped_ordering).reset_index(drop=True)
for genre in sorted(outer_df["list_name"].unique()):
    curr_genre_df = outer_df.query(f"list_name == '{genre}'")
    print(f"[TOP {cut} {mode.upper()} CHARTS] - {genre}\n")
    (
        curr_genre_df[['name', 'stars']]
            .reset_index(drop=True)
            .reset_index()
            .rename(columns={"index": "RANK"})
            .assign(RANK=lambda x: x.RANK + 1)
            .apply(lambda x: print(f"\t {int(x['RANK'])}. {x['name']} ({(int(x['stars']))} stars)"), axis=1)
    )
    print("")

### Top 20 worst by genre

In [ ]:
mode = "worst"
cut = 20

if mode == "best":
    ascending_order = [False, False]
elif mode == "worst":
    ascending_order = [True, True]
def grouped_ordering(g):
    return g.sort_values(by=["stars", "relative_rank"], ascending=ascending_order)[["year_month", "year", "month", "name", "stars", "relative_rank", "list_name"]].head(cut)
outer_df = cards_df.groupby("list_name", group_keys=False).apply(grouped_ordering).reset_index(drop=True)
for genre in sorted(outer_df["list_name"].unique()):
    curr_genre_df = outer_df.query(f"list_name == '{genre}'")
    print(f"[TOP {cut} {mode.upper()} CHARTS] - {genre}\n")
    (
        curr_genre_df[['name', 'stars']]
            .reset_index(drop=True)
            .reset_index()
            .rename(columns={"index": "RANK"})
            .assign(RANK=lambda x: x.RANK + 1)
            .apply(lambda x: print(f"\t {int(x['RANK'])}. {x['name']} ({(int(x['stars']))} stars)"), axis=1)
    )
    print("")